In [7]:
%pwd

'f:\\zizo\\RTKCorrection\\research'

In [8]:
import os
os.chdir("../")
%pwd

'f:\\zizo\\RTKCorrection'

In [9]:
import pandas as pd
df1 = pd.read_csv("data/input.txt",sep='\t')
df1.head()

,Time_seconds,SatelliteID,C1C,L1C,D1C,S1C
0,15568.004,4,2.183553e+07,1.147465e+08,-2062.373,44
1,15568.004,7,2.403299e+07,1.262942e+08,1800.175,46
2,15568.004,8,2.177553e+07,1.144312e+08,975.683,46
3,15568.004,9,2.159508e+07,1.134829e+08,541.655,49
4,15568.004,27,2.263255e+07,1.189348e+08,-1535.173,46


In [11]:
df1['SatelliteID'].unique()

array([ 4,  7,  8,  9, 27, 16, 30, 14, 21,  2, 22, 17, 13, 19])

In [12]:
df1.groupby("Time_seconds")["SatelliteID"].value_counts()[:20]

Time_seconds  SatelliteID
15568.004     4              2
              7              2
              8              2
              9              2
              27             2
15569.004     4              2
              7              2
              8              2
              9              2
              27             2
15570.004     4              2
              7              2
              8              2
              9              2
              27             2
15571.004     4              2
              7              2
              8              2
              9              2
              27             2
Name: count, dtype: int64

In [13]:
df1["GPST_seconds"] = (df1['Time_seconds'].round()).astype(int)
df1.drop('Time_seconds',axis=1,inplace=True)
df1.head()

,SatelliteID,C1C,L1C,D1C,S1C,GPST_seconds
0,4,2.183553e+07,1.147465e+08,-2062.373,44,15568
1,7,2.403299e+07,1.262942e+08,1800.175,46,15568
2,8,2.177553e+07,1.144312e+08,975.683,46,15568
3,9,2.159508e+07,1.134829e+08,541.655,49,15568
4,27,2.263255e+07,1.189348e+08,-1535.173,46,15568


In [14]:
# checking unique repetation number for GPST_seconds
df1["GPST_seconds"].value_counts().unique()

array([17, 16, 15, 14, 13, 12, 11, 10,  9,  8,  7,  5,  1])

In [15]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112082 entries, 0 to 112081
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   SatelliteID   112082 non-null  int64  
 1   C1C           112082 non-null  float64
 2   L1C           95855 non-null   float64
 3   D1C           112082 non-null  float64
 4   S1C           112082 non-null  int64  
 5   GPST_seconds  112082 non-null  int64  
dtypes: float64(3), int64(3)
memory usage: 5.1 MB


In [16]:
df1.isna().sum()

SatelliteID         0
C1C                 0
L1C             16227
D1C                 0
S1C                 0
GPST_seconds        0
dtype: int64

In [17]:
(df1['L1C'].isna().sum()/len(df1))*100

np.float64(14.477793044378224)

# Data Preprocessing
## 1) Data handling

In [24]:
from sklearn.impute import KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)
imputed_data = knn_imputer.fit_transform(df1)
imputed_df = pd.DataFrame(imputed_data)
imputed_df.head()

,0,1,2,3,4,5
0,4.0,2.183553e+07,1.147465e+08,-2062.373,44.0,15568.0
1,7.0,2.403299e+07,1.262942e+08,1800.175,46.0,15568.0
2,8.0,2.177553e+07,1.144312e+08,975.683,46.0,15568.0
3,9.0,2.159508e+07,1.134829e+08,541.655,49.0,15568.0
4,27.0,2.263255e+07,1.189348e+08,-1535.173,46.0,15568.0


In [26]:
imputed_df.columns = ["SatelliteID", "C1C", "L1C", "D1C", "S1C", "GPST_seconds"]
imputed_df['GPST_seconds'] = imputed_df['GPST_seconds'].astype(int)
imputed_df.head()

,SatelliteID,C1C,L1C,D1C,S1C,GPST_seconds
0,4.0,2.183553e+07,1.147465e+08,-2062.373,44.0,15568
1,7.0,2.403299e+07,1.262942e+08,1800.175,46.0,15568
2,8.0,2.177553e+07,1.144312e+08,975.683,46.0,15568
3,9.0,2.159508e+07,1.134829e+08,541.655,49.0,15568
4,27.0,2.263255e+07,1.189348e+08,-1535.173,46.0,15568


In [27]:
imputed_df.isna().sum()

SatelliteID     0
C1C             0
L1C             0
D1C             0
S1C             0
GPST_seconds    0
dtype: int64

In [28]:
imputed_df.to_csv("data/input_preprocesed.csv", index=False)

# Perparing Output Data

In [ ]:
def extract_pos_coordinates(filepath, output_csv=None):
    extracted_rows = []

    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            # Skip metadata and comments
            if not line or line.startswith('%'):
                continue
            # Extract the first 4 comma-separated columns
            parts = [p.strip() for p in line.split(',')]
            if len(parts) >= 4:
                gpst = parts[0]
                lat = float(parts[1])
                lon = float(parts[2])
                height = float(parts[3])
                extracted_rows.append([gpst, lat, lon, height])

    # Optional: write to CSV
    if output_csv:
        import csv
        with open(output_csv, 'w', newline='') as f_out:
            writer = csv.writer(f_out)
            writer.writerow(['GPST', 'Latitude', 'Longitude', 'Height'])
            writer.writerows(extracted_rows)

    return extracted_rows

# Example usage
pos_file_path = 'F:\zizo\RTKCorrection\data\output.pos'
output = extract_pos_coordinates(pos_file_path, output_csv='data/corrected_positions.csv')

# Show a few lines
for row in output[:5]:
    print(row)


['2024/02/14 04:19:28.00', 27.682077363, 85.319258109, 1274.4728]
['2024/02/14 04:19:29.00', 27.682035126, 85.319265362, 1255.8659]
['2024/02/14 04:19:30.00', 27.682052033, 85.319250831, 1264.7933]
['2024/02/14 04:19:31.00', 27.682024398, 85.319245599, 1249.076]
['2024/02/14 04:19:32.00', 27.681998721, 85.31920834, 1265.345]


In [30]:
df2 = pd.read_csv('corrected_positions.csv')
df2.head()

,GPST,Latitude,Longitude,Height
0,2024/02/14 04:19:28.00,27.682077,85.319258,1274.4728
1,2024/02/14 04:19:29.00,27.682035,85.319265,1255.8659
2,2024/02/14 04:19:30.00,27.682052,85.319251,1264.7933
3,2024/02/14 04:19:31.00,27.682024,85.319246,1249.0760
4,2024/02/14 04:19:32.00,27.681999,85.319208,1265.3450


In [31]:
df2['GPST_seconds'] = pd.to_datetime(df2['GPST']).dt.hour * 3600 + \
                     pd.to_datetime(df2['GPST']).dt.minute * 60 + \
                     pd.to_datetime(df2['GPST']).dt.second

df2.drop('GPST', axis=1, inplace=True)
df2.head()

,Latitude,Longitude,Height,GPST_seconds
0,27.682077,85.319258,1274.4728,15568
1,27.682035,85.319265,1255.8659,15569
2,27.682052,85.319251,1264.7933,15570
3,27.682024,85.319246,1249.0760,15571
4,27.681999,85.319208,1265.3450,15572


In [32]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7891 entries, 0 to 7890
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Latitude      7891 non-null   float64
 1   Longitude     7891 non-null   float64
 2   Height        7891 non-null   float64
 3   GPST_seconds  7891 non-null   int32  
dtypes: float64(3), int32(1)
memory usage: 215.9 KB
